In [1]:
import numpy as np

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 

import requests 
from pandas.io.json import json_normalize 

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium 

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


<h3>Q1 : Create DataFrame with pandas</h3>

In [7]:
import pandas as pd

import requests

from bs4 import BeautifulSoup


req = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")

soup = BeautifulSoup(req.content,'lxml')

table = soup.find_all('table')[0]

df = pd.read_html(str(table))

neighborhood=pd.DataFrame(df[0])

neighborhood.head(10)


,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,NaN
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,"Malvern, Rouge"


In [8]:
neighborhood.shape

(180, 3)

In [9]:
neighborhood2= neighborhood[neighborhood.Borough!='Not assigned']
neighborhood2.reset_index(drop=True)
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhood2['Borough'].unique()),
        neighborhood2.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighborhoods.


<h3>Q2 : get the latitude and the longitude coordinates of each neighborhood<\h3>

In [12]:
df = pd.read_csv('https://cocl.us/Geospatial_data')

In [13]:
df1 = neighborhood2.merge(df, on='Postal Code')

In [14]:
df1.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [15]:
df1.shape[0]

103

<h3>Q3 : Explore and cluster the neighborhoods in Toronto<\h3>

In [16]:
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [18]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df1['Latitude'], df1['Longitude'], df1['Borough'], df1['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

<h4>work with only boroughs that contain the word Toronto<\h4>

In [21]:
East_Toronto = df1[df1.Borough=='East Toronto'].reset_index(drop=True)
East_Toronto.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M7Y,East Toronto,Business reply mail Processing Centre,43.662744,-79.321558


In [22]:
address = 'East Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Downtown Toronto are 43.6534817, -79.3839347.


In [24]:
# create map of East Toronto using latitude and longitude values
map_ET = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(East_Toronto['Latitude'], East_Toronto['Longitude'], East_Toronto['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_ET)  
    
map_ET

In [25]:
CLIENT_ID = 'GFWJDGGPY1Z3HK0LCHDTTM5CZO3SYOAU5VFT0IH0ZXRM4RCS'
CLIENT_SECRET = 'JWGXIALXWPRADXP1OTL2B0EPQVEWQBHGFWL5DGPDWOQXIWQB'
VERSION = '20180605'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: GFWJDGGPY1Z3HK0LCHDTTM5CZO3SYOAU5VFT0IH0ZXRM4RCS
CLIENT_SECRET:JWGXIALXWPRADXP1OTL2B0EPQVEWQBHGFWL5DGPDWOQXIWQB


In [26]:
East_Toronto.loc[0, 'Neighborhood']

'The Beaches'

In [27]:
neighborhood_latitude = East_Toronto.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = East_Toronto.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = East_Toronto.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of The Beaches are 43.67635739999999, -79.2930312.


In [28]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=GFWJDGGPY1Z3HK0LCHDTTM5CZO3SYOAU5VFT0IH0ZXRM4RCS&client_secret=JWGXIALXWPRADXP1OTL2B0EPQVEWQBHGFWL5DGPDWOQXIWQB&v=20180605&ll=43.67635739999999,-79.2930312&radius=500&limit=100'

In [29]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ec8cd4a78a484001b364f49'},
 'response': {'headerLocation': 'The Beaches',
  'headerFullLocation': 'The Beaches, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 4,
  'suggestedBounds': {'ne': {'lat': 43.680857404499996,
    'lng': -79.28682091449052},
   'sw': {'lat': 43.67185739549999, 'lng': -79.29924148550948}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bd461bc77b29c74a07d9282',
       'name': 'Glen Manor Ravine',
       'location': {'address': 'Glen Manor',
        'crossStreet': 'Queen St.',
        'lat': 43.67682094413784,
        'lng': -79.29394208780985,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.67682094413784,
          'lng': -79.29394208780985}],
        'distanc

In [30]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [31]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Glen Manor Ravine,Trail,43.676821,-79.293942
1,The Big Carrot Natural Food Market,Health Food Store,43.678879,-79.297734
2,Grover Pub and Grub,Pub,43.679181,-79.297215
3,Upper Beaches,Neighborhood,43.680563,-79.292869


In [32]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

4 venues were returned by Foursquare.


In [33]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [34]:
East_Toronto_venues = getNearbyVenues(names=East_Toronto['Neighborhood'],
                                   latitudes=East_Toronto['Latitude'],
                                   longitudes=East_Toronto['Longitude']
                                  )



The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Business reply mail Processing Centre


In [35]:
print(East_Toronto_venues.shape)
East_Toronto_venues.head()

(122, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West, Riverdale",43.679557,-79.352188,MenEssentials,43.677820,-79.351265,Cosmetics Shop


In [36]:
East_Toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Business reply mail Processing Centre,17,17,17,17,17,17
"India Bazaar, The Beaches West",19,19,19,19,19,19
Studio District,40,40,40,40,40,40
The Beaches,4,4,4,4,4,4
"The Danforth West, Riverdale",42,42,42,42,42,42


In [37]:
print('There are {} uniques categories.'.format(len(East_Toronto_venues['Venue Category'].unique())))

There are 67 uniques categories.


<h4>Analyze each Neighborhood<\h4>

In [39]:
# one hot encoding
East_Toronto_onehot = pd.get_dummies(East_Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
East_Toronto_onehot['Neighborhood'] = East_Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [East_Toronto_onehot.columns[-1]] + list(East_Toronto_onehot.columns[:-1])
East_Toronto_onehot = East_Toronto_onehot[fixed_columns]

East_Toronto_onehot.head()

,Yoga Studio,American Restaurant,Auto Workshop,Bakery,Bank,Bar,Bookstore,Brewery,Bubble Tea Shop,Burrito Place,Butcher,Café,Caribbean Restaurant,Cheese Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Comic Shop,Convenience Store,Cosmetics Shop,Coworking Space,Dessert Shop,Diner,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gastropub,Gay Bar,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Health Food Store,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Juice Bar,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Middle Eastern Restaurant,Movie Theater,Neighborhood,Park,Pet Store,Pizza Place,Pub,Recording Studio,Restaurant,Sandwich Place,Seafood Restaurant,Skate Park,Smoke Shop,Spa,Stationery Store,Steakhouse,Sushi Restaurant,Thai Restaurant,Trail,Wine Bar
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,The Beaches,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"The Danforth West, Riverdale",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [40]:
East_Toronto_onehot.shape

(122, 67)

In [41]:
East_Toronto_grouped = East_Toronto_onehot.groupby('Neighborhood').mean().reset_index()
East_Toronto_grouped

,Neighborhood,Yoga Studio,American Restaurant,Auto Workshop,Bakery,Bank,Bar,Bookstore,Brewery,Bubble Tea Shop,Burrito Place,Butcher,Café,Caribbean Restaurant,Cheese Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Comic Shop,Convenience Store,Cosmetics Shop,Coworking Space,Dessert Shop,Diner,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gastropub,Gay Bar,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Health Food Store,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Juice Bar,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Middle Eastern Restaurant,Movie Theater,Park,Pet Store,Pizza Place,Pub,Recording Studio,Restaurant,Sandwich Place,Seafood Restaurant,Skate Park,Smoke Shop,Spa,Stationery Store,Steakhouse,Sushi Restaurant,Thai Restaurant,Trail,Wine Bar
0,Business reply mail Processing Centre,0.00000,0.00000,0.058824,0.00000,0.000,0.000,0.00000,0.058824,0.00000,0.058824,0.058824,0.00000,0.00000,0.000,0.000,0.000000,0.000,0.058824,0.000,0.00000,0.000,0.00000,0.000,0.058824,0.058824,0.000000,0.000,0.00000,0.00000,0.000000,0.058824,0.058824,0.00,0.000,0.000000,0.00000,0.000000,0.058824,0.00,0.000000,0.00000,0.000000,0.00000,0.000,0.058824,0.000000,0.00000,0.000,0.000000,0.058824,0.000000,0.058824,0.000000,0.058824,0.058824,0.000000,0.000,0.058824,0.058824,0.00000,0.000,0.000000,0.000000,0.000,0.00000,0.000
1,"India Bazaar, The Beaches West",0.00000,0.00000,0.000000,0.00000,0.000,0.000,0.00000,0.052632,0.00000,0.052632,0.000000,0.00000,0.00000,0.000,0.000,0.052632,0.000,0.000000,0.000,0.00000,0.000,0.00000,0.000,0.000000,0.105263,0.052632,0.000,0.00000,0.00000,0.000000,0.000000,0.000000,0.00,0.000,0.000000,0.00000,0.052632,0.000000,0.00,0.052632,0.00000,0.052632,0.00000,0.000,0.000000,0.052632,0.00000,0.000,0.052632,0.052632,0.052632,0.052632,0.052632,0.000000,0.052632,0.052632,0.000,0.000000,0.000000,0.00000,0.000,0.052632,0.052632,0.000,0.00000,0.000
2,Studio District,0.02500,0.05000,0.000000,0.05000,0.025,0.025,0.02500,0.050000,0.00000,0.000000,0.000000,0.10000,0.00000,0.025,0.025,0.075000,0.025,0.000000,0.025,0.00000,0.025,0.00000,0.025,0.000000,0.000000,0.000000,0.025,0.00000,0.00000,0.000000,0.000000,0.000000,0.05,0.025,0.000000,0.00000,0.000000,0.025000,0.00,0.025000,0.00000,0.025000,0.00000,0.025,0.000000,0.000000,0.00000,0.025,0.000000,0.025000,0.025000,0.000000,0.000000,0.000000,0.000000,0.025000,0.025,0.000000,0.000000,0.00000,0.025,0.000000,0.000000,0.025,0.00000,0.025
3,The Beaches,0.00000,0.00000,0.000000,0.00000,0.000,0.000,0.00000,0.000000,0.00000,0.000000,0.000000,0.00000,0.00000,0.000,0.000,0.000000,0.000,0.000000,0.000,0.00000,0.000,0.00000,0.000,0.000000,0.000000,0.000000,0.000,0.00000,0.00000,0.000000,0.000000,0.000000,0.00,0.000,0.000000,0.00000,0.000000,0.000000,0.25,0.000000,0.00000,0.000000,0.00000,0.000,0.000000,0.000000,0.00000,0.000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.00000,0.000,0.000000,0.000000,0.000,0.25000,0.000
4,"The Danforth West, Riverdale",0.02381,0.02381,0.000000,0.02381,0.000,0.000,0.02381,0.023810,0.02381,0.000000,0.000000,0.02381,0.02381,0.000,0.000,0.071429,0.000,0.000000,0.000,0.02381,0.000,0.02381,0.000,0.000000,0.000000,0.000000,0.000,0.02381,0.02381,0.047619,0.000000,0.000000,0.00,0.000,0.214286,0.02381,0.000000,0.000000,0.00,0.047619,0.02381,0.071429,0.02381,0.000,0.000000,0.023810,0.02381,0.000,0.000000,0.000000,0.000000,0.023810,0.023810,0.000000,0.047619,0.000000,0.000,0.000000,0.000000,0.02381,0.000,0.000000,0.000000,0.000,0.02381,0.000


In [42]:
East_Toronto_grouped.shape

(5, 67)

<h4>print each neighborhood along with the top 5 most common venues<\h4>

In [44]:
num_top_venues = 5

for hood in East_Toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = East_Toronto_grouped[East_Toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Business reply mail Processing Centre----
                  venue  freq
0  Gym / Fitness Center  0.06
1         Garden Center  0.06
2        Farmers Market  0.06
3                  Park  0.06
4           Pizza Place  0.06


----India Bazaar, The Beaches West----
                  venue  freq
0  Fast Food Restaurant  0.11
1                   Pub  0.05
2         Movie Theater  0.05
3                  Park  0.05
4             Pet Store  0.05


----Studio District----
                 venue  freq
0                 Café  0.10
1          Coffee Shop  0.08
2            Gastropub  0.05
3               Bakery  0.05
4  American Restaurant  0.05


----The Beaches----
               venue  freq
0              Trail  0.25
1                Pub  0.25
2  Health Food Store  0.25
3        Yoga Studio  0.00
4          Juice Bar  0.00


----The Danforth West, Riverdale----
                    venue  freq
0        Greek Restaurant  0.21
1      Italian Restaurant  0.07
2             Coffee Shop  0.07
3 

In [45]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [50]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = East_Toronto_grouped['Neighborhood']

for ind in np.arange(East_Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(East_Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Business reply mail Processing Centre,Recording Studio,Brewery,Butcher,Park,Burrito Place,Pizza Place,Light Rail Station,Comic Shop,Restaurant,Farmers Market
1,"India Bazaar, The Beaches West",Fast Food Restaurant,Pub,Gym,Coffee Shop,Ice Cream Shop,Italian Restaurant,Liquor Store,Fish & Chips Shop,Movie Theater,Park
2,Studio District,Café,Coffee Shop,Gastropub,American Restaurant,Bakery,Brewery,Convenience Store,Ice Cream Shop,Gym / Fitness Center,Gay Bar
3,The Beaches,Trail,Pub,Health Food Store,Farmers Market,Convenience Store,Cosmetics Shop,Coworking Space,Dessert Shop,Diner,Fast Food Restaurant
4,"The Danforth West, Riverdale",Greek Restaurant,Coffee Shop,Italian Restaurant,Furniture / Home Store,Restaurant,Ice Cream Shop,Grocery Store,Juice Bar,Indian Restaurant,Yoga Studio


<h3>Kmeans Clustering<\h3>

In [51]:
# set number of clusters
kclusters = 5

East_Toronto_grouped_clustering = East_Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(East_Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 2, 4, 1, 0], dtype=int32)

In [52]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

East_Toronto_merged = East_Toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
East_Toronto_merged = East_Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

East_Toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,1,Trail,Pub,Health Food Store,Farmers Market,Convenience Store,Cosmetics Shop,Coworking Space,Dessert Shop,Diner,Fast Food Restaurant
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Furniture / Home Store,Restaurant,Ice Cream Shop,Grocery Store,Juice Bar,Indian Restaurant,Yoga Studio
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,2,Fast Food Restaurant,Pub,Gym,Coffee Shop,Ice Cream Shop,Italian Restaurant,Liquor Store,Fish & Chips Shop,Movie Theater,Park
3,M4M,East Toronto,Studio District,43.659526,-79.340923,4,Café,Coffee Shop,Gastropub,American Restaurant,Bakery,Brewery,Convenience Store,Ice Cream Shop,Gym / Fitness Center,Gay Bar
4,M7Y,East Toronto,Business reply mail Processing Centre,43.662744,-79.321558,3,Recording Studio,Brewery,Butcher,Park,Burrito Place,Pizza Place,Light Rail Station,Comic Shop,Restaurant,Farmers Market


In [53]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(East_Toronto_merged['Latitude'], East_Toronto_merged['Longitude'], East_Toronto_merged['Neighborhood'], East_Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<h3>Examine Clusters<\h3>

In [54]:
East_Toronto_merged.loc[East_Toronto_merged['Cluster Labels'] == 0, East_Toronto_merged.columns[[1] + list(range(5, East_Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,East Toronto,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Furniture / Home Store,Restaurant,Ice Cream Shop,Grocery Store,Juice Bar,Indian Restaurant,Yoga Studio


In [56]:
East_Toronto_merged.loc[East_Toronto_merged['Cluster Labels'] == 1, East_Toronto_merged.columns[[1] + list(range(5, East_Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,1,Trail,Pub,Health Food Store,Farmers Market,Convenience Store,Cosmetics Shop,Coworking Space,Dessert Shop,Diner,Fast Food Restaurant


In [57]:
East_Toronto_merged.loc[East_Toronto_merged['Cluster Labels'] == 2, East_Toronto_merged.columns[[1] + list(range(5, East_Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,East Toronto,2,Fast Food Restaurant,Pub,Gym,Coffee Shop,Ice Cream Shop,Italian Restaurant,Liquor Store,Fish & Chips Shop,Movie Theater,Park


In [58]:
East_Toronto_merged.loc[East_Toronto_merged['Cluster Labels'] == 3, East_Toronto_merged.columns[[1] + list(range(5, East_Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,East Toronto,3,Recording Studio,Brewery,Butcher,Park,Burrito Place,Pizza Place,Light Rail Station,Comic Shop,Restaurant,Farmers Market


In [59]:
East_Toronto_merged.loc[East_Toronto_merged['Cluster Labels'] == 4, East_Toronto_merged.columns[[1] + list(range(5, East_Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,East Toronto,4,Café,Coffee Shop,Gastropub,American Restaurant,Bakery,Brewery,Convenience Store,Ice Cream Shop,Gym / Fitness Center,Gay Bar
